In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
team_stats = pd.read_csv('../footy_data/championship_teams_2022.csv')
games = pd.read_csv('../footy_data/championship_games_2022.csv')

In [3]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [4]:
games = games.loc[games['status'] == 'complete']
#We only want games that have been completed

In [5]:
test_prediction_games = games[73:84]
# I don't think I will need this

In [6]:
games = games[0:72]
#Completed Matches
games.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,over/under
0,1628275500,Aug 06 2021 - 6:45pm,complete,NaN,AFC Bournemouth,West Bromwich Albion,Dean Whitestone,1,0.0,0.0,...,3.40,3.10,1.33,1.85,3.15,6.00,1.80,2.05,Vitality Stadium (Bournemouth- Dorset),1
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Blackburn Rovers,Swansea City,David Webb,1,0.0,0.0,...,3.25,3.60,1.38,2.15,3.65,7.25,1.95,1.87,Ewood Park (Blackburn- Lancashire),1
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bristol City,Blackpool,Andy Davies,1,0.0,0.0,...,3.35,3.45,1.39,2.10,3.45,6.75,1.95,1.91,Ashton Gate (Bristol),0
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cardiff City,Barnsley,Steve Martin,1,0.0,0.0,...,3.20,3.00,1.45,2.25,3.85,7.50,1.95,1.91,Cardiff City Stadium (Cardiff (Caerdydd)),0
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Derby County,Huddersfield Town,Geoff Eltringham,1,0.0,0.0,...,3.35,2.80,1.43,2.15,3.70,7.00,1.95,1.87,Pride Park Stadium (Derby),0


In [7]:
team_stats['actual_xg'] = ""
team_stats['actual_xga'] = ""
#Creating average xG and xGA for each team myself because FootyStats is too lazy to put it correctly in the spreadsheet

In [8]:
ful_var = games.loc[games['home_team_name'] == "Fulham"] 
ful_var = ful_var.append(games.loc[games['away_team_name'] == "Fulham"])
ful_var

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,over/under
10,1628425800,Aug 08 2021 - 12:30pm,complete,NaN,Fulham,Middlesbrough,Keith Stroud,1,0.0,0.0,...,3.50,4.75,1.43,2.15,3.80,7.25,2.00,1.83,Craven Cottage (London),0
43,1629554400,Aug 21 2021 - 2:00pm,complete,NaN,Fulham,Hull City,Tim Robinson,4,1.0,3.0,...,4.75,8.75,1.29,1.83,2.95,5.50,2.20,1.67,Craven Cottage (London),0
51,1630159200,Aug 28 2021 - 2:00pm,complete,NaN,Fulham,Stoke City,Jarred Gillett,5,2.0,2.0,...,3.40,4.60,1.48,2.30,4.10,8.25,2.15,1.71,Craven Cottage (London),1
16,1628949600,Aug 14 2021 - 2:00pm,complete,NaN,Huddersfield Town,Fulham,Dean Whitestone,2,0.0,0.0,...,3.40,1.91,1.44,2.25,3.90,7.75,2.10,1.77,John Smith's Stadium (Huddersfield- West Yorks...,1
27,1629225900,Aug 17 2021 - 6:45pm,complete,NaN,Millwall,Fulham,Geoff Eltringham,3,1.0,3.0,...,2.95,2.28,1.42,2.25,3.90,7.75,2.00,1.83,The Den (London),1
63,1631368800,Sep 11 2021 - 2:00pm,complete,NaN,Blackpool,Fulham,John Busby,6,0.0,3.0,...,4.15,1.61,1.33,1.91,3.20,6.00,2.05,1.80,Bloomfield Road (Blackpool- Lancashire),0


In [9]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [10]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]



In [11]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [12]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [13]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 

#Reading in stats

# Read in stats for new DF we'll use for testing

In [14]:
test_prediction_games['home_xg'] = ""
test_prediction_games['away_xg'] = ""
test_prediction_games['home_xg_against'] = ""
test_prediction_games['away_xg_against'] = ""
test_prediction_games['home_total_corners'] = ""
test_prediction_games['away_total_corners'] = ""

# Creating new columns where our data will go

In [15]:
def fill_stats_new_test(team):
    test_prediction_games['home_xg'] = np.where(test_prediction_games['home_team_name'] == team, xg_stats[team], test_prediction_games['home_xg'])
    test_prediction_games['away_xg'] = np.where(test_prediction_games['away_team_name'] == team, xg_stats[team], test_prediction_games['away_xg'])
    test_prediction_games['home_xg_against'] = np.where(test_prediction_games['home_team_name'] == team, xga_stats[team], test_prediction_games['home_xg_against'])
    test_prediction_games['away_xg_against'] = np.where(test_prediction_games['away_team_name'] == team, xga_stats[team], test_prediction_games['away_xg_against'])
    test_prediction_games['home_total_corners'] = np.where(test_prediction_games['home_team_name'] == team, corner_stats[team], test_prediction_games['home_total_corners'])
    test_prediction_games['away_total_corners'] = np.where(test_prediction_games['away_team_name'] == team, corner_stats[team], test_prediction_games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [131]:
fill_stats_new_test('Fulham')
fill_stats_new_test('Blackburn Rovers')
fill_stats_new_test('West Bromwich Albion')
fill_stats_new_test('Queens Park Rangers')
fill_stats_new_test('AFC Bournemouth')
fill_stats_new_test('Middlesbrough')
fill_stats_new_test('Barnsley')
fill_stats_new_test('Millwall')
fill_stats_new_test('Sheffield United')
fill_stats_new_test('Reading')
fill_stats_new_test('Cardiff City')
fill_stats_new_test('Nottingham Forest')
fill_stats_new_test('Hull City')
fill_stats_new_test('Blackpool')
fill_stats_new_test('Luton Town')
fill_stats_new_test('Stoke City')
fill_stats_new_test('Swansea City')
fill_stats_new_test('Derby County')
fill_stats_new_test('Huddersfield Town')
fill_stats_new_test('Preston North End')
fill_stats_new_test('Coventry City')
fill_stats_new_test('Peterborough United')
fill_stats_new_test('Birmingham City')
fill_stats_new_test('Bristol City')

# Some more EDA

In [132]:
games.corr()['over/under'].sort_values(ascending=False)[0:20]

over/under                        1.000000
total_goal_count                  0.826399
home_team_goal_count              0.647750
away_team_goal_count              0.598754
home_team_shots_on_target         0.537915
total_goals_at_half_time          0.484470
away_team_shots_on_target         0.461219
team_a_xg                         0.443857
home_team_shots                   0.412773
away_team_red_cards               0.387743
team_b_xg                         0.377192
away_team_goal_count_half_time    0.367785
home_team_red_cards               0.355257
home_team_goal_count_half_time    0.337149
home_team_possession              0.326314
away_team_shots                   0.321160
away_team_possession              0.320207
home_team_first_half_cards        0.266847
away_team_fouls                   0.266080
home_team_fouls                   0.263804
Name: over/under, dtype: float64

# Now let's do some modeling

This will be a 'normal' model in which we use train_test_split to divide up the data

In [133]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = games[features]
y = games['over/under']

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [135]:
logreg = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()

In [136]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [137]:
preds = logreg.predict(X_test)

In [138]:
accuracy_score(y_test, preds)

0.6470588235294118

In [139]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [140]:
preds = knn.predict(X_test)

In [141]:
accuracy_score(y_test, preds)

0.5294117647058824

In [142]:
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [143]:
preds = dt.predict(X_test)

In [144]:
accuracy_score(y_test, preds)

0.47058823529411764

In [145]:
#decision trees

In [148]:
y_test

73    0
0     1
58    0
22    1
12    1
70    0
10    0
18    1
4     0
65    0
40    1
47    0
35    1
81    0
55    0
54    0
33    1
Name: over/under, dtype: int32

Test the test set while it is trained on the original train set?

In [116]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X = test_prediction_games[features]
y = test_prediction_games['over/under']

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 42)

In [ ]:
logreg.predict()

In [151]:
games[0:5]['home_team_name']

0     AFC Bournemouth
1    Blackburn Rovers
2        Bristol City
3        Cardiff City
4        Derby County
Name: home_team_name, dtype: object

# Trying to set the train and test sets myself without train_test_split

In [169]:
features = ['home_xg', 'away_xg', 'home_xg_against', 'away_xg_against']
X_train = games[0:72][features]
X_test = games[72:84][features]
y_train = games[0:72]['over/under']
y_test = games[72:84]['over/under']

In [170]:
X_test

,home_xg,away_xg,home_xg_against,away_xg_against
72,1.02,1.36,0.97,1.21
73,0.79,0.59,1.26,1.15
74,0.77,0.99,1.48,1.15
75,1.34,1.04,0.82,0.93
76,0.99,0.53,1.54,1.39
77,1.74,0.8,0.7,1.16
78,1.16,1.8,0.81,0.72
79,1.03,1.05,0.97,1.2
80,1.56,1.38,0.95,0.83
81,0.8,1.15,1.18,0.99


In [171]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [172]:
preds = logreg.predict(X_test)

In [173]:
accuracy_score(y_test, preds)

0.16666666666666666

In [174]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [175]:
preds = knn.predict(X_test)

In [176]:
accuracy_score(y_test, preds)

0.4166666666666667

In [177]:
preds

array([1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1])

Next need to add something to track accuracy from week to week